<a href="https://colab.research.google.com/github/francescoS01/Bayesian-Network/blob/main/bayesian_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### **1. Bayesian node class**
**1.1 attributi di istanza**:<br>
nome del nodo `node_name`, possibili valori che può assumere un nodo `possibile_value`


**1.2 get and set**:<br>
Sono stati definiti i metodi get e set per poter leggere e scrivere i valori delle variabili del nodo.
<br><br>
**1.2 value generate:** <br>
Questo metodo peremtte di settare il valore corrente (self.current_value) del nodo conoscendo tutti i suoi genitori e seguendo le probabilità definite nella tabella cpt (definita come array di oggetti, verrà mostrata successivamente in dettaglio). Quindi, ad esempio, considerando un nodo X con due genitori Y e Z che hanno rispettivamente valori vy e vz, questo metodo imposterà il valore del nodo X in base alle probabilità che X assuma un determinato valore vi: P(X=vi | Y=yi, Z=zi). Il metodo assume che la teballa cpt di ogni nodo rispetti la seguente proprietà delle reti bayesiane:  $$ \sum_{vi \in \text{possible value}} P(X=vi \mid \text{parents value}) = 1 $$
<br>
Vediamo le tre fasi che compongono il metodo. Per farlo consideriamo l'esempio precedente, quindi di analizzare il nodo X che assume valori $vi$ in {X possible value} e i suoi due genitori Y e Z con i rispettivi valori vy e vz:
- viene creato una dizionario che contiene i nomi dei genitori con i loro valori: {Y:yi, Z:zi}.
- vengono cercati all'interno della tebella cpt gli oggetti (uno per ogni possibile valore che può assumere X) che assumono { Y:yi, Z:zi } e da questi vengono estratte le informazioni necessarie, cioè che valore assume il nodo e con quale probabilità per quei dati valori dei genitori. Tutte queste infromazioni vengono via via inserite in un oggetto e alla fine di questa fase si sarà creato il seguente dizonario: {v1:probv1, ..., vn:probn} dove le v sono i valori che può assumere il nodo X. (OSS. somma delle probvi = 1)
- viene generato un numero casuale tra 0 e 1. Questo numero determina quale valore il nodo X assumerà basandosi sulle probabilità assegnate a ciascun valore. Ad esempio, se il numero casuale cade in un intervallo di probabilità più alto, il nodo X assumerà il corrispondente valore associato a quell'intervallo.

In [6]:
import random
from typing import List

class BayesianNode:
    def __init__(self, node_name, possible_value, cpt, current_value=None, children=[], parents=[]):
        self.node_name = node_name
        self.possible_value = possible_value
        self.children = children
        self.parents = parents
        self.cpt = cpt
        self.current_value = current_value  
        self.children_update() 

    def children_update(self):
        for parent in self.parents:
            parent.set_children(self)

    def set_children(self, new_child: 'BayesianNode'):
        self.children.append(new_child)

    def set_current_state(self, value): 
        self.current_value = value

    def get_name(self):
        return self.node_name

    def get_parents(self):
        return self.parents
    
    def get_children(self): 
        return self.children
     
    def get_current_value(self): 
        return self.current_value
    
    def get_cpt(self): 
        return self.cpt 
    
    def value_generate(self):
        # create a dictionary with paretns and them value
        parent_value_dict = {}
        for parent in self.parents:
            parent_name = parent.get_name()
            value = parent.get_current_value()
            parent_value_dict[parent_name] = value

        # create a dictionary of probability of self node knowing parents and them values 
        probability_distribution = {}
        for value in self.possible_value:
            new = parent_value_dict.copy()
            new[self.node_name] = value
            for dict in self.cpt: 
                new['prob'] = dict['prob']
                if dict == new:
                    probability_distribution[value] = dict['prob']
                    
        # following the probility, extract one value of the self node 
        numero_random = random.random() # 0-1 
        accumulate = 0
        for key, prob in probability_distribution.items():
            accumulate += prob
            if accumulate >= numero_random:
                self.current_value = key
                return key

### **2. Network node class**
La classe network contiene un solo attributo, un array di oggetti ti tipo Nodo.  <br>
`topological_sort`: ordina la lista dei nodi (self_nodes) in ordine topologico, così da poterla utilizzare succissevamente per fare sampling. <br><br>
**2.1 sampling_create** <br>
viene eseguito un ordine topologico dei nodi e successivamente per ognuni nodo viene chiamato il metodo value_generate (definito nella classe Nodo). Tutti i nodi con il proprio valore generato vengono messi in un oggetto sampling che verrà restituito.


In [7]:
class Network:
    def __init__(self, nodes:List[BayesianNode]):
        self.net_nodes = nodes

    def  add_node(self, node: BayesianNode):
        if not isinstance(node, BayesianNode):
            raise TypeError("Input must be of type 'BayesianNode'")
        else:
            self.net_nodes.append(node)

    def topological_sort(self):
        visited = []
        parents_count = {node: 0 for node in self.net_nodes}
        for node in self.net_nodes:
            for child in node.children:
                parents_count[child] += 1
        no_parents = [node for node in self.net_nodes if parents_count[node] == 0]
        while no_parents:
            node = no_parents.pop(0)
            visited.append(node)
            for child in node.children:
                parents_count[child] -= 1
                if parents_count[child] == 0:
                    no_parents.append(child)
        self.net_nodes = visited 
        return visited
    
    def sampling_create(self): 
        nodes = self.topological_sort()
        sampling = {}
        for node in nodes: 
            node_name = node.get_name()
            node_value = node.value_generate()
            sampling[node_name] = node_value
            node.set_current_state(node_value)
        return sampling

### **3. Node creation**
In questa sezione vengono creati gli oggetti di tipo nodi con tutte le variabili di istanza necessarie, nome del nodo, genitori, etc. O <br><br>
**3.1 CPT**<br>
La tabella cpt è stata creata una per ogni nodo come un array di dizionari. Per ogni valore possibile del nodo in considerazione vengono creati un numero di dizionari (all'interno dell'array) pari a tutte le possibili combinazioni che si hanno considerando i valori che possono assumere i genitori. Riprendendo il solito esempio del nodo X con due genitori Y e Z vediamo un generico dizionario contenuto nella cpt del nodo X: {X:vx, Y:vy, Z:vz, 'prob':p} dove p indica la probalità che X assuma valore vx sapendo sapendo che Y e Z assumono rispettivamente valore vy, vz.  
<br><br>
**3.2 Struttura bayesian network**<br>
![Img](https://drive.google.com/uc?export=view&id=1Ocse1AFURWeeybF5vT8wmxm8i-DiC9m3)



In [8]:
#OSS: the first values of CPT is always the  first states of the variable itself 

# NODE 1: Nutrition node
nutr_possible_value = ["good", "not good"]
nutr_name ="nutrition"
parents = []
children = []
current_value = None
cpt = [                                                                                                 
    {nutr_name: 'good', 'prob':0.5},
    {nutr_name: 'not good', 'prob':0.5}]
nutr_node = BayesianNode(nutr_name, nutr_possible_value , cpt, current_value, children, parents)

# NODE 2: physical exercise node
pysicalex_possible_value = ['good', 'not good']
pysicalex_name = 'pysical exercise'
parents = []
children = []
current_value = None
cpt = [                                                                                                 
    {pysicalex_name: 'good', 'prob':0.6},
    {pysicalex_name: 'not good', 'prob':0.4}]
pysicalex_node = BayesianNode(pysicalex_name, pysicalex_possible_value , cpt, current_value, children, parents)

# NODE 3: healt node
healt_possibile_value = ['good', 'not good']
healt_name = 'healt'
parents = [nutr_node, pysicalex_node]
children = []
current_value = None
nutr_name = nutr_node.get_name()
pysicalex_name = pysicalex_node.get_name()
cpt = [                                                                                                 
    {healt_name: 'good', nutr_name: 'good', pysicalex_name: 'good', 'prob':0.8},
    {healt_name: 'good', nutr_name: 'good', pysicalex_name: 'not good', 'prob':0.7},
    {healt_name: 'good', nutr_name: 'not good', pysicalex_name: 'good', 'prob':0.6}, 
    {healt_name: 'good', nutr_name: 'not good', pysicalex_name: 'not good', 'prob':0.3},
    {healt_name: 'not good', nutr_name: 'good', pysicalex_name: 'good', 'prob':0.2},
    {healt_name: 'not good', nutr_name: 'good', pysicalex_name: 'not good', 'prob':0.3},
    {healt_name: 'not good', nutr_name: 'not good', pysicalex_name: 'good', 'prob':0.4},
    {healt_name: 'not good', nutr_name: 'not good', pysicalex_name: 'not good', 'prob':0.7}]
healt_node = BayesianNode(healt_name, healt_possibile_value, cpt, current_value, children, parents)

# NODE 4: stress node
stress_possibile_value = ['high', 'not high']
stress_name = 'stress'
parents = [healt_node]
children = []
current_value = None
healt_name = healt_node.get_name()
cpt = [                                                                                                 
    {stress_name: 'high', healt_name: 'good', 'prob':0.4},
    {stress_name: 'high', healt_name: 'not good', 'prob':0.8},
    {stress_name: 'not high', healt_name: 'good', 'prob':0.6}, 
    {stress_name: 'not high', healt_name: 'not good', 'prob':0.2}]
stress_node = BayesianNode(stress_name, stress_possibile_value, cpt, current_value, children, parents)

# NODE 5: recovery node
recovery_possibile_value = ['good', 'not good']
recovery_name = 'recovery'
parents = [stress_node]
children = []
current_value = None
stress_name = stress_node.get_name()
cpt = [                                                                                                 
    {recovery_name: 'good', stress_name: 'high', 'prob':0.2},
    {recovery_name: 'good', stress_name: 'not high', 'prob':0.7},
    {recovery_name: 'not good', stress_name: 'high', 'prob':0.8}, 
    {recovery_name: 'not good', stress_name: 'not high', 'prob':0.3}]
recovery_node = BayesianNode(recovery_name, recovery_possibile_value, cpt, current_value, children, parents)

# NODE 6: mood node
mood_possibile_value = ['good', 'not good']
mood_name = 'mood'
parents = [stress_node]
children = []
current_value = None
stress_name = stress_node.get_name()
cpt = [                                                                                                 
    {mood_name: 'good', stress_name: 'high', 'prob':0.2},
    {mood_name: 'good', stress_name: 'not high', 'prob':0.7},
    {mood_name: 'not good', stress_name: 'high', 'prob':0.8}, 
    {mood_name: 'not good', stress_name: 'not high', 'prob':0.3}]
mood_node = BayesianNode(mood_name, mood_possibile_value, cpt, current_value, children, parents)

# NODE 7: energy node
energy_possibile_value = ['high', 'not high']
energy_name = 'energy'
parents = [recovery_node, nutr_node]
children = []
current_value = None
recovery_name = recovery_node.get_name()
nutr_name = nutr_node.get_name()
cpt = [                                                                                                 
    {energy_name: 'high', nutr_name: 'good', recovery_name: 'good', 'prob':0.9},
    {energy_name: 'high', nutr_name: 'good', recovery_name: 'not good', 'prob':0.5},
    {energy_name: 'high', nutr_name: 'not good', recovery_name: 'good', 'prob':0.6}, 
    {energy_name: 'high', nutr_name: 'not good', recovery_name: 'not good', 'prob':0.1},
    {energy_name: 'not high', nutr_name: 'good', recovery_name: 'good', 'prob':0.1},
    {energy_name: 'not high', nutr_name: 'good', recovery_name: 'not good', 'prob':0.4},
    {energy_name: 'not high', nutr_name: 'not good', recovery_name: 'good', 'prob':0.9},
    {energy_name: 'not high', nutr_name: 'not good', recovery_name: 'not good', 'prob':0.9}]
energy_node = BayesianNode(energy_name, energy_possibile_value, cpt, current_value, children, parents)

# NODE 8: productivity node
prod_possibile_value = ['high', 'not high']
prod_name = 'productivity'
parents = [energy_node, mood_node]
children = []
current_value = None
energy_name = energy_node.get_name()
mood_name = mood_node.get_name()
cpt = [                                                                                                 
    {prod_name: 'high', energy_name: 'high', mood_name: 'good', 'prob':0.8},
    {prod_name: 'high', energy_name: 'high', mood_name: 'not good', 'prob':0.6},
    {prod_name: 'high', energy_name: 'not high', mood_name: 'good', 'prob':0.6}, 
    {prod_name: 'high', energy_name: 'not high', mood_name: 'not good', 'prob':0.2},
    {prod_name: 'not high', energy_name: 'high', mood_name: 'good', 'prob':0.2},
    {prod_name: 'not high', energy_name: 'high', mood_name: 'not good', 'prob':0.4},
    {prod_name: 'not high', energy_name: 'not high', mood_name: 'good', 'prob':0.4},
    {prod_name: 'not high', energy_name: 'not high', mood_name: 'not good', 'prob':0.8}]
productivity_node = BayesianNode(prod_name, prod_possibile_value, cpt, current_value, children, parents)

# NODE 9: wellness node
wellness_possibile_value = ['high', 'not high']
wellness_name = 'wellness'
parents = [pysicalex_node, mood_node]
children = []
current_value = None
pysicalex_name = pysicalex_node.get_name()
mood_name = mood_node.get_name()
cpt = [                                                                                                 
    {wellness_name: 'high', pysicalex_name: 'good', mood_name: 'good', 'prob':0.8},
    {wellness_name: 'high', pysicalex_name: 'good', mood_name: 'not good', 'prob':0.6},
    {wellness_name: 'high', pysicalex_name: 'not good', mood_name: 'good', 'prob':0.6}, 
    {wellness_name: 'high', pysicalex_name: 'not good', mood_name: 'not good', 'prob':0.2},
    {wellness_name: 'not high', pysicalex_name: 'good', mood_name: 'good', 'prob':0.2},
    {wellness_name: 'not high', pysicalex_name: 'good', mood_name: 'not good', 'prob':0.4},
    {wellness_name: 'not high', pysicalex_name: 'not good', mood_name: 'good', 'prob':0.4},
    {wellness_name: 'not high', pysicalex_name: 'not good', mood_name: 'not good', 'prob':0.8}]
wellness_node = BayesianNode(wellness_name, wellness_possibile_value, cpt, current_value, children, parents)

#### **4. Sampling creation**

In [9]:
# ------------------------- NET TEST --------------------------------
net = Network([mood_node, nutr_node, healt_node, energy_node, pysicalex_node, stress_node, recovery_node, productivity_node , wellness_node])
for i in range(1, 11): # mi fai un for di 10?
    x = net.sampling_create()
    print(x)

{'nutrition': 'good', 'pysical exercise': 'not good', 'healt': 'good', 'stress': 'not high', 'recovery': 'good', 'mood': 'not good', 'energy': 'not high', 'wellness': 'not high', 'productivity': 'not high'}
mammt
{'nutrition': 'good', 'pysical exercise': 'good', 'healt': 'good', 'stress': 'high', 'recovery': 'good', 'mood': 'not good', 'energy': 'high', 'wellness': 'not high', 'productivity': 'high'}
mammt
{'nutrition': 'good', 'pysical exercise': 'not good', 'healt': 'good', 'stress': 'not high', 'recovery': 'not good', 'mood': 'not good', 'energy': 'high', 'wellness': 'not high', 'productivity': 'not high'}
mammt
{'nutrition': 'good', 'pysical exercise': 'not good', 'healt': 'not good', 'stress': 'not high', 'recovery': 'not good', 'mood': 'good', 'energy': 'high', 'wellness': 'not high', 'productivity': 'not high'}
mammt
{'nutrition': 'good', 'pysical exercise': 'good', 'healt': 'good', 'stress': 'high', 'recovery': 'good', 'mood': 'not good', 'energy': 'high', 'wellness': 'not high